# Testing the null hypothesis H0: There is no difference between temperature climatologies from 1951-1980 and 1989-2018!

We use monthly mean data from station observations that are part of the GHCN data base. 
The code an extension of the code developed for the confidence interval calculations (see unit 7). The main purpose of this coding activity: Conduct a formal t-test and find stations and months for which we can reject the null hypothesis. In other words we want to explore where in NY and during which season we have the strongest data support for a significant warming.

Note: In this notebook we read a list of station id numbers (and additional 'metadata' information) from a text file that has all of the NY stations listed. Many stations do not have complete time series over the whole time period 1951-2018, so those are discarded. 




## 1. Code development

### 1.1 Importing our own set of support functions

In this script you notice we do not define the function for downloading the data from the ACIS server. 

Instead, we can separate the function definitions from our Notebook and import the functions with the same syntax that we use to import packages like _numpy_ or _scipy.stats_. This importing of Python code from separate files is known as import of _modules_. (The file is pure Python code and must have the extension *.py*.) 

Our Python script is called support.py (see GitHub repository unit8, download the script file, and upload it here into directory *unit8*). Note the ending must be .py for this Python code text file. This is referred to as 'import of modules'.

**Download the file support.py from GitHub (see folder unit8) and upload it into the same folder where you have this notebook file. The file must be named _support.py_ !**


Note: Packages are more complex, consisting of entire folders and subfolders with Python code. So modules are much simpler to maintain and a good first start to get your useful functions organized.




### 1.2 Importing all packages and our own module as spt

And check what we imported and how the functions work.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from scipy import stats


import support as spt
# new: use function help() to see the content of the imported module and the doc-string information
help(spt)


### Reading the file with the NY station data information (metadata)

In [ ]:
# Get station list from FTP site
sidlist,latlist,lonlist,elevlist,namelist=spt.get_station_list(sid_code="USW")
print ("number of stations starting with 'USW' (Airport stations)",len(sidlist))

### 1.3 Preparing the data for statistical calculations

<P style="background-color:lightgreen;color:black;font-size:130%">
<BR>
Check which stations have complete data, and  form a final list of stations to use.
<BR>
<BR>
</P>


**The code below is one 'basic' level approach to deal with storing all station data
that have complete time series without any missing values.**

Here we use a simple approach: 
- Access all the station data from the ACIS server. 
- For each station, the downloaded data is checked for missing values.
- If data are complete for the whole period (1951 to 2018, see variable year1, year2 above), then the station metdata information (station id, latitude, longitude, elevation, and full station name) is appended to new lists.

Once we have screened our station data we can do our calculation and plotting for any of those stations by selecting one of the station id names.

In [ ]:
varname="avgt"
year1,year2= 1951,2018

In [ ]:
print ("data complete between %4d and %4d?" %(year1,year2))
complete_sid=[]
complete_lat=[]
complete_lon=[]
complete_elev=[]
complete_name=[]
i=0
for station_id in sidlist:
    x,y=spt.get_stationdata_monthly(station_id,varname,startyear=year1,endyear=year2)
    if len(x)==0:
        pass
    else:
        x=np.array(x)
        y=np.array(y)
        # check for completeness in data (no NAN allowed)
        iuse=np.logical_and(x>dt.datetime(1950,12,31),x<dt.datetime(2019,1,1))
        imiss=np.isnan(y[iuse])
        if any(imiss): # NAN values detected in time series
            pass
            #print (3*"-"+"station: "+station_id+" variable "+varname+" has NAN values.")
        else: # no NAN detected
            print (3*"+"+"station: "+station_id+" variable "+varname+" is complete.")
            complete_sid.append(station_id)
            complete_lat.append(latlist[i])
            complete_lon.append(lonlist[i])
            complete_elev.append(elevlist[i])
            complete_name.append(namelist[i])
    i=i+1
nstation=len(complete_sid)
print ("number of stations, number of stations with complete data", i,nstation)


In [ ]:
print("stations that are available")
for i,sid in enumerate(complete_sid): # new syntax
    print (i,sid,complete_name[i])


<P style="background-color:lightgreen;color:black;font-size:130%">
<BR>
Read data for one of the stations stored in the list of complete stations.
<BR>
<BR>
</P>




In [ ]:
station_index=0 #
print (complete_sid[station_index])
print (complete_name[station_index])
x,y=spt.get_stationdata_monthly(complete_sid[station_index],\
                            varname,startyear=year1,endyear=year2)
x=np.array(x)
y=np.array(y)


<P style="background-color:lightgreen;color:black;font-size:130%">
<BR>
Select the data from x and y for the two 30 climate period: 1951-1980 and 1989-2018
<BR>
<BR>
</P>
    
Assign the results to new variables and check with np.shape the dimensions and size of the data array. You should have 360 data left in the arrays. 


In [ ]:
# create first subsample
iclim1=np.logical_and(x>dt.datetime(1950,12,31),x<dt.datetime(1981,1,1))
xclim1=x[iclim1]
yclim1=y[iclim1]
print("check climatology data sample 1:")
print(xclim1[0]," - ",xclim1[-1])
print(np.shape(yclim1))

In [ ]:
# create second subsample
iclim2=np.logical_and(x>dt.datetime(1988,12,31),x<dt.datetime(2019,1,1))
xclim2=x[iclim2]
yclim2=y[iclim2]
print("check climatology data sample 2:")
print(xclim2[0]," - ",xclim2[-1])
print(np.shape(yclim2))

<P style="background-color:lightgreen;color:black;font-size:130%">
<BR>
Calculate the mean and standard deviation for each month (using np.reshape)
<BR>
<BR>
</P>
    


In [ ]:
yhelp1=np.reshape(yclim1,newshape=(30,12))
yhelp2=np.reshape(yclim2,newshape=(30,12))

ymean1=np.mean(yhelp1,axis=0)
ymean2=np.mean(yhelp2,axis=0)
ystd1=np.std(yhelp1,axis=0)
ystd2=np.std(yhelp2,axis=0)

<P style="background-color:lightgreen;color:black;font-size:130%">
<BR>
Confidence intervals
<BR>
<BR>
</P>

In [ ]:
# sample size here is 30 years, but we should make it a data-dependent value.
#n1, n2 = 30, 30
n1= np.shape(yhelp1)[0] # rows are the years
n2= np.shape(yhelp2)[0] # rows are the years (could be different from sample 1) 
df1=n1-1
df2=n2-1
alpha=0.95 # for both samples
tint1=stats.t.interval(alpha,df1) # returns lists
tint2=stats.t.interval(alpha,df2) # returns lists
# arrays for the upper and lower confidence intervals
cf1=np.zeros(shape=[12,2])
cf2=np.zeros(shape=[12,2]) # a true independent copy 
for i in range(12):
    cf1[i,:]=np.array(tint1)*ystd1[i]/np.sqrt(n1)
    cf2[i,:]=np.array(tint2)*ystd2[i]/np.sqrt(n2)
    i+=1
    

## 2. Results

<P style="background-color:purple;color:gold;font-size:130%">
<BR>
Task 1: Present the climatologies in a graph or two that make it easy to see the differences between the two periods, and how much the confidence intervals overlap.
<BR>
<BR>
</P>
    
- Tip: Check the function plt.errorbar

<P style="background-color:purple;color:gold;font-size:130%">
<BR>
Task 2: Calculate the t-test with the help of the function provide in Practical Statistics Appendix A. Choose a either a winter month, and a summer month, where you expect to reject the null hypothesis and obtain the smallest p-value.
<BR>
<BR>
</P>
    
Tip: Check out the help or google examples for the the application of the function Scipy *stats.ttest_ind*.
(see link below)

Print out the essential information: 
- the difference in the mean, 
- the t-statistic, the p-value, 
- the test decision, 
- and interpretation of the sign if the difference (sign of the t-statistic).



## 3 Summary and conclusion

Some comments/remarks here.


---
### References

- [Introduction to import of modules](https://www.programiz.com/python-programming/modules)
- [Function ttest_ind from scipy.stats](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)
- [Welch's form of the t-test](https://en.wikipedia.org/wiki/Welch%27s_t-test) (ttest_ind is calculating this test statistic)